# Message and Communication

An agent in AGNext can react to, send, and publish messages,
and messages are the only means through which agents can communicate
with each other.

## Messages

Messages are serializable objects, they can be defined using:

- A subclass of Pydantic's {py:class}`pydantic.BaseModel`, or
- A dataclass

For example:

In [2]:
from dataclasses import dataclass


@dataclass
class TextMessage:
    content: str
    source: str


@dataclass
class ImageMessage:
    url: str
    source: str

```{note}
Messages are purely data, and should not contain any logic.
```

### Message Handlers

When an agent receives a message the runtime will invoke the agent's message handler
({py:meth}`~agnext.core.Agent.on_message`) which should implement the agents message handling logic.
If this message cannot be handled by the agent, the agent should raise a
{py:class}`~agnext.core.exceptions.CantHandleException`.

For convenience, the {py:class}`~agnext.components.TypeRoutedAgent` base class
provides the {py:meth}`~agnext.components.message_handler` decorator
for associating message types with message handlers,
so developers do not need to implement the {py:meth}`~agnext.core.Agent.on_message` method.

For example, the following type-routed agent responds to `TextMessage` and `ImageMessage`
using different message handlers:

In [4]:
from agnext.application import SingleThreadedAgentRuntime
from agnext.components import TypeRoutedAgent, message_handler
from agnext.core import AgentId, MessageContext


class MyAgent(TypeRoutedAgent):
    @message_handler
    async def on_text_message(self, message: TextMessage, ctx: MessageContext) -> None:
        print(f"Hello, {message.source}, you said {message.content}!")

    @message_handler
    async def on_image_message(self, message: ImageMessage, ctx: MessageContext) -> None:
        print(f"Hello, {message.source}, you sent me {message.url}!")

Create the agent runtime and register the agent (see [Agent and Agent Runtime](agent-and-agent-runtime.ipynb)):

In [6]:
runtime = SingleThreadedAgentRuntime()
await runtime.register("my_agent", lambda: MyAgent("My Agent"))
agent = AgentId("my_agent", "default")

Test this agent with `TextMessage` and `ImageMessage`.

In [7]:
runtime.start()
await runtime.send_message(TextMessage(content="Hello, World!", source="User"), agent)
await runtime.send_message(ImageMessage(url="https://example.com/image.jpg", source="User"), agent)
await runtime.stop_when_idle()

Hello, User, you said Hello, World!!
Hello, User, you sent me https://example.com/image.jpg!


## Communication

There are two types of communication in AGNext:

- **Direct communication**: An agent sends a direct message to another agent.
- **Broadcast communication**: An agent publishes a message to all agents in the same namespace.

### Direct Communication

To send a direct message to another agent, within a message handler use
the {py:meth}`agnext.core.BaseAgent.send_message` method,
from the runtime use the {py:meth}`agnext.core.AgentRuntime.send_message` method.
Awaiting calls to these methods will return the return value of the
receiving agent's message handler.

```{note}
If the invoked agent raises an exception while the sender is awaiting,
the exception will be propagated back to the sender.
```

#### Request/Response

Direct communication can be used for request/response scenarios,
where the sender expects a response from the receiver.
The receiver can respond to the message by returning a value from its message handler.
You can think of this as a function call between agents.

For example, consider the following type-routed agent:

In [17]:
from dataclasses import dataclass

from agnext.application import SingleThreadedAgentRuntime
from agnext.components import TypeRoutedAgent, message_handler
from agnext.core import MessageContext


@dataclass
class Message:
    content: str


class InnerAgent(TypeRoutedAgent):
    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"Hello from inner, {message.content}")


class OuterAgent(TypeRoutedAgent):
    def __init__(self, description: str, inner_agent_type: str):
        super().__init__(description)
        self.inner_agent_id = AgentId(inner_agent_type, self.id.key)

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> None:
        print(f"Received message: {message.content}")
        # Send a direct message to the inner agent and receves a response.
        response = await self.send_message(Message(f"Hello from outer, {message.content}"), self.inner_agent_id)
        print(f"Received inner response: {response.content}")

Upone receving a message, the `OuterAgent` sends a direct message to the `InnerAgent` and receives
a message in response.

We can test these agents by sending a `Message` to the `OuterAgent`.

In [18]:
runtime = SingleThreadedAgentRuntime()
await runtime.register("inner_agent", lambda: InnerAgent("InnerAgent"))
await runtime.register("outer_agent", lambda: OuterAgent("OuterAgent", "InnerAgent"))
runtime.start()
outer = AgentId("outer_agent", "default")
await runtime.send_message(Message(content="Hello, World!"), outer)
await runtime.stop_when_idle()

Received message: Hello, World!
Received inner response: Hello from inner, Hello from outer, Hello, World!


Both outputs are produced by the `OuterAgent`'s message handler, however the second output is based on the response from the `InnerAgent`.

### Broadcast Communication

Broadcast communication is effectively the publish/subscribe model.
As part of the base agent ({py:class}`~agnext.core.BaseAgent`) implementation,
it must advertise the message types that
it would like to receive when published ({py:attr}`~agnext.core.AgentMetadata.subscriptions`).
If one of these messages is published, the agent's message handler will be invoked.

The key difference between direct and broadcast communication is that broadcast
communication cannot be used for request/response scenarios.
When an agent publishes a message it is one way only, it cannot receive a response
from any other agent, even if a receiving agent sends a response.

```{note}
An agent receiving a message does not know if it is handling a published or direct message.
So, if a response is given to a published message, it will be thrown away.
```

To publish a message to all agents in the same namespace,
use the {py:meth}`agnext.core.BaseAgent.publish_message` method.
This call must still be awaited to allow the runtime to deliver the message to all agents,
but it will always return `None`.
If an agent raises an exception while handling a published message,
this will be logged but will not be propagated back to the publishing agent.

The following example shows a `BroadcastingAgent` that publishes a message
upong receiving a message. A `ReceivingAgent` that prints the message
it receives.

In [20]:
from agnext.application import SingleThreadedAgentRuntime
from agnext.components import TypeRoutedAgent, message_handler
from agnext.core import MessageContext, TopicId


class BroadcastingAgent(TypeRoutedAgent):
    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> None:
        # Publish a message to all agents in the same namespace.
        assert ctx.topic_id is not None
        await self.publish_message(
            Message(f"Publishing a message: {message.content}!"), topic_id=TopicId("deafult", self.id.key)
        )


class ReceivingAgent(TypeRoutedAgent):
    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> None:
        print(f"Received a message: {message.content}")

Sending a direct message to the `BroadcastingAgent` will result in a message being published by
the `BroadcastingAgent` and received by the `ReceivingAgent`.

In [21]:
from agnext.components import TypeSubscription

runtime = SingleThreadedAgentRuntime()
await runtime.register("broadcasting_agent", lambda: BroadcastingAgent("Broadcasting Agent"))
await runtime.register("receiving_agent", lambda: ReceivingAgent("Receiving Agent"))
await runtime.add_subscription(TypeSubscription("default", "broadcasting_agent"))
await runtime.add_subscription(TypeSubscription("default", "receiving_agent"))
runtime.start()
await runtime.send_message(Message("Hello, World!"), AgentId("broadcasting_agent", "default"))
await runtime.stop()

Received a message: Publishing a message: Hello, World!!


To publish a message to all agents outside of an agent handling a message,
the message should be published via the runtime with the
{py:meth}`agnext.core.AgentRuntime.publish_message` method.

In [22]:
# Replace send_message with publish_message in the above example.

runtime = SingleThreadedAgentRuntime()
await runtime.register("broadcasting_agent", lambda: BroadcastingAgent("Broadcasting Agent"))
await runtime.register("receiving_agent", lambda: ReceivingAgent("Receiving Agent"))
await runtime.add_subscription(TypeSubscription("default", "broadcasting_agent"))
await runtime.add_subscription(TypeSubscription("default", "receiving_agent"))
runtime.start()
await runtime.publish_message(Message("Hello, World! From the runtime!"), topic_id=TopicId("default", "default"))
await runtime.stop_when_idle()

Received a message: Hello, World! From the runtime!
Received a message: Publishing a message: Hello, World! From the runtime!!


The first output is from the `ReceivingAgent` that received a message published
by the runtime. The second output is from the `ReceivingAgent` that received
a message published by the `BroadcastingAgent`.

```{note}
If an agent publishes a message type for which it is subscribed it will not
receive the message it published. This is to prevent infinite loops.
```